# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание проекта

Имеется дата-сет с параметрами поведения клиентов телеком-оператора.
В дата-сете 3214 строк и 5 колонок, обозначающих: кол-во звонков, длительность звонков, кол-во сообщений, потраченные Мб трафика и тариф ("Ультра" и "Смарт").\
Так-как перед нами стоит задача машинного обучения, предполагается полное отсутствие пропусков.\
По имеющимся данным необходимо построить систему рекомендации подходящих тарифов клиентам оператора.\
Примерный план выполнения задачи:
- Импорт необходимых библиотек
- Откр. и ознакомление с дата-фреймом
- Разбитие данных на три выборки (тренировочную, тестовую и валидационную) в пропорции 6:2:2
- Исследование моделей и подбор лучших гиперпараметров для каждой
- Исследование наиболее точной модели на тестовой выборке
- Проверка лучшей модели на адекватность
- Вывод

Начнем с импорта необходимых библиотек.

In [6]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

## Откройте и изучите файл

 ### Откроем csv-файл, используя оба источника на случай технических неисправностей.

In [7]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

### Изучим основные хар-ки датафрейма.

In [8]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [9]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Наблюдаем близкие значения средней и медианы, что говорит о небольшом кол-ве выбросов и аномальных значений.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропуски в данных отсутствуют, следуем дальше.

In [11]:
df[df['is_ultra'] == 1].count()

calls       985
minutes     985
messages    985
mb_used     985
is_ultra    985
dtype: int64

In [12]:
df[df['is_ultra'] == 0].count()

calls       2229
minutes     2229
messages    2229
mb_used     2229
is_ultra    2229
dtype: int64

Наблюдаем значительную разницу в объеме выборок.\
Пользователей тарифа **ultra** в 2 с небольшим раза меньше, чем пользователей тарифа **smart**.

In [13]:
df.corr().style.background_gradient(cmap='coolwarm')

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


## Разбейте данные на выборки

### Разобьем нашу выборку на обучающую, тестовую и валидационную в пропорциях 6:2:2.

In [14]:
df_train, df_test =train_test_split(df,stratify=df['is_ultra'], test_size=0.4, random_state=12345)
df_test, df_val=train_test_split(df_test,stratify=df_test['is_ultra'], test_size=0.5, random_state=12345)

### Создадим переменные для признаков и целевого признака.

In [15]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']
features_val = df_val.drop('is_ultra', axis=1)
target_val = df_val['is_ultra']

In [16]:
print(df_train.shape, df_test.shape, df_val.shape)

(1928, 5) (643, 5) (643, 5)


## Исследуйте модели

### Начнем с решающего дерева

Пройдем циклом по модели "Решающее дерево" автоматически меняя гиперпараметры.

In [17]:
%%time

best_model = None
best_dt_accuracy = 0

for depth in range(1,6):
    for split in range(2,12):
        for leaf in range(2,12):
            dt = DecisionTreeClassifier(random_state=12345, 
                                        max_depth= depth, 
                                        min_samples_split= split, 
                                        min_samples_leaf=leaf)
            dt.fit(features_train, target_train)
            dt.predict = dt.predict(features_val)
            dt_accuracy = accuracy_score(target_val, dt.predict)
            if dt_accuracy > best_dt_accuracy:
                best_model = dt
                best_dt_accuracy = dt_accuracy
                
print(f'Наиболее точные пар-ры: {best_model}')
print(f'Наивысшая точность решающего дерева: {best_dt_accuracy}')
print()

Наиболее точные пар-ры: DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, random_state=12345)
Наивысшая точность решающего дерева: 0.8102643856920684

CPU times: total: 1.39 s
Wall time: 3.83 s


### Случайный лес

Также пройдем циклом по модели "Случайный лес" автоматически меняя гиперпараметры.

In [18]:
best_rf_model = None
best_rf_accuracy = 0

for estim in tqdm(range(3,50)):
    for depth in range(1,6):
        for split in range(2,12):
            for leaf in range(2,12):
                rf = RandomForestClassifier(random_state = 12345,
                                            n_estimators = estim,
                                            max_depth = depth,
                                            min_samples_split= split,
                                            min_samples_leaf=leaf)
                rf.fit(features_train, target_train)
                rf.predict = rf.predict(features_val)
                rf_accuracy = accuracy_score(target_val, rf.predict)
                if rf_accuracy > best_rf_accuracy:
                    best_rf_model = rf
                    best_rf_accuracy = rf_accuracy
                
print(f'Наиболее точные пар-ры: {best_rf_model}')
print(f'Наивысшая точность Случайного леса: {best_rf_accuracy}')

  0%|          | 0/47 [00:00<?, ?it/s]

### Логистическая регрессия

Построим модель Логистической регрессии по аналогии с предыдущими моделями.

In [ ]:
%%time

best_lr_model = None
best_lr_accuracy = 0

for penalty in tqdm(['l2']):
    for solver in tqdm(['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']):
        lr = LogisticRegression(random_state = 12345,
                                penalty = penalty,
                                solver = solver)
        lr.fit(features_train, target_train)
        lr.predict = lr.predict(features_val)
        lr_accuracy = accuracy_score(target_val, lr.predict)
                
        if lr_accuracy > best_lr_accuracy:
                best_lr_model = lr
                best_lr_accuracy = lr_accuracy
                
print(f'Наиболее точные пар-ры: {best_lr_model}')
print(f'Наивысшая точность Случайного леса: {best_lr_accuracy}')
print()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Наиболее точные пар-ры: LogisticRegression(random_state=12345)
Наивысшая точность Случайного леса: 0.7465007776049767

CPU times: user 228 ms, sys: 7.96 ms, total: 236 ms
Wall time: 271 ms


<div class="alert alert-info">
 Bonus <s>track</s> models.
Из любопытства попробовал еще 2 модели.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>



Успех 👍:



Ну и отлично.  Только метод лежащих соседей - это метрическая модель, для неё нужно масштабировать данные, а чтобы не было утечки данных надо использовать pipeline.  В общем тут целая история




</div>


### K-Neighbors.

In [ ]:
%%time

best_kn_model = None
best_kn_accuracy = 0

for neighbors in tqdm(range(1, 50)):
    for weights in['uniform', 'distance']:
        for leaf in range(2, 100):
            kn = KNeighborsClassifier(n_neighbors = neighbors,
                                     weights = weights,
                                     leaf_size = leaf)
            kn.fit(features_train, target_train)
            kn.predict = kn.predict(features_val)
            kn_accuracy = accuracy_score(target_val, kn.predict)
            
            if kn_accuracy > best_kn_accuracy:
                best_kn_model = kn
                best_kn_accuracy = kn_accuracy
                
print(f'Наиболее точные пар-ры: {best_kn_model}')
print(f'Наивысшая точность Ближайших соседей: {best_kn_accuracy}')
print()

  0%|          | 0/49 [00:00<?, ?it/s]

Наиболее точные пар-ры: KNeighborsClassifier(leaf_size=2, n_neighbors=12)
Наивысшая точность Ближайших соседей: 0.7729393468118196

CPU times: user 2min 31s, sys: 401 ms, total: 2min 31s
Wall time: 2min 33s


### SVC (Метод опорных векторов).

In [ ]:
best_svc_model = None
best_svc_accuracy = 0

for C in tqdm(range(1,20)):
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        svc = SVC(random_state = 12345,
                      C = C,
                      kernel = kernel)
        svc.fit(features_train, target_train)
        svc.predict = svc.predict(features_val)
        svc_accuracy = accuracy_score(target_val, svc.predict)
        
        if svc_accuracy > best_svc_accuracy:
                best_svc_model = svc
                best_svc_accuracy = svc_accuracy
                
print(f'Наиболее точные пар-ры: {best_svc_model}')
print(f'Наивысшая точность Метода опорных векторов: {best_svc_accuracy}')
print()

  0%|          | 0/19 [00:00<?, ?it/s]

Наиболее точные пар-ры: SVC(C=12, random_state=12345)
Наивысшая точность Метода опорных векторов: 0.7527216174183515



## Проверьте модель на тестовой выборке

Наилучшей моделью оказалась модель Случайного леса.
Точность прогноза на валидационной выборке составила: 82 %
При использовании гиперпараметров: max_depth=5, min_samples_leaf=2, min_samples_split=7,n_estimators=28, random_state=12345

Продолжим исследование этой модели, но уже на тестовой выборке.

In [ ]:
best_rf_model = RandomForestClassifier(random_state=12345,
                                 n_estimators=28,
                                 min_samples_split=7,
                                 min_samples_leaf=2,
                                 max_depth=5)
best_rf_model.fit(pd.concat([features_train, features_val]), pd.concat([target_train, target_val]))
best_rf_model.predict = best_rf_model.predict(features_test)
best_rf_model_accuracy = accuracy_score(target_test, best_rf_model.predict)

print(f'Наивысшая точность Случайного леса: {best_rf_model_accuracy}')

Наивысшая точность Случайного леса: 0.8118195956454122


## (бонус) Проверьте модели на адекватность

Для проверки на адекватность решено использовать DummyClassifier.\
Если настроенная модель Случайного леса покажет точность выше DummyClassifier, то наша модель будет считаться адекватной (вменяемой).

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_test, target_test)
dummy_clf.predict = dummy_clf.predict(features_test)
dummy_clf_accuracy = accuracy_score(target_test, dummy_clf.predict)
dummy_clf_accuracy

0.6936236391912908

## Вывод

Так-как дата-сет был заранее подготовлен, нам не понадобилась предобработка данных.\
Было изучено 5 моделей: Решающее дерево (Decision tree), Случайный лес (Random Forest), Логистическая регрессия (Logistic Regression), К-ближайших соседей (K-Neighbors), Метод опорных векторов (SVC, Support Vector Classification).\
Для каждой модели были предложены лучшие гиперпараметры.\
Лучшей оказалась модель Случайного Леса (Random Forest) с точностью 82% на валидационной и 83% на тестовой выборках.\
Модель случайного леса умпешно прошла проверку на адекватность Dummy Classifier из библиотеки SK Learn со стратегией most_frequent.\
На этом исследование можно считать успешно завершенным.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
